In [1]:
import torch
import numpy as np
from model_parameters import ModelParameters
from tqdm.notebook import tqdm

# Data Loading

In [2]:
model_parameters = ModelParameters()
x_train, x_test, y_train, y_test = model_parameters.get_htru_2()

# Model definition

In [3]:
class ClassicCNN(torch.nn.Module):
    def __init__(self, num_features):
        super(ClassicCNN, self).__init__()
        self.conv1d = torch.nn.Conv1d(in_channels=num_features, out_channels=128, kernel_size=1)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(128, 64)
        self.fc2 = torch.nn.Linear(64, 1)

    def forward(self, x):
        x = torch.reshape(x, (-1, 8,1))
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        x = torch.sigmoid(x)
        x = x.squeeze(1)
        return x

# Training CNN

In [4]:
def train_network(model,optimizer,criterion,x_train,y_train,x_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(x_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train.float())
        
        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()
        
        output_test = model(x_test)
        loss_test = criterion(output_test,y_test.float())

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

for i in tqdm(range(model_parameters.num_runs)):
    num_epochs = model_parameters.max_num_epochs
    train_losses = np.zeros(num_epochs)
    test_losses  = np.zeros(num_epochs)
    model = ClassicCNN(model_parameters.num_features)

    learning_rate = 0.01
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

    balanced_x_train, balanced_y_train = model_parameters.sub_select_dataset(x_train, y_train, model_parameters.training_samples, balanced=True)
    sub_selected_x_test, sub_selected_y_test = model_parameters.sub_select_dataset(x_test, y_test, model_parameters.testing_samples)

    balanced_x_train, sub_selected_x_test, balanced_y_train, sub_selected_y_test = model_parameters.torch_convertion(balanced_x_train, sub_selected_x_test, balanced_y_train, sub_selected_y_test)

    train_network(model,optimizer,criterion, balanced_x_train,balanced_y_train, sub_selected_x_test,sub_selected_y_test,num_epochs,train_losses,test_losses)

    # Predict the test set
    output_test = model(sub_selected_x_test)
    predicted_test = (output_test > 0.5).float()

    # Calculate the scores
    model_parameters.append_score(sub_selected_y_test, predicted_test)

  0%|          | 0/3 [00:00<?, ?it/s]

# Prediction

In [5]:
# Print the scores
for metric, values in model_parameters.scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")

Accuracy: 0.933 ± 0.019
Balanced_accuracy: 0.912 ± 0.023
Recall: 0.887 ± 0.042
Specificity: 0.938 ± 0.021
Precision: 0.619 ± 0.072
Npv: 0.987 ± 0.004
Gmean: 0.912 ± 0.024
Informedness: 0.825 ± 0.046
